# Cleaning of Sales and Product Masterfile Data

Notebook by Dominic Dayta, for Aroodai and Lean Retail Labs.

In [ ]:
import pandas as pd
import numpy as np

I need this function for later, to check if a given string is a float or not.

In [ ]:
def is_number(n):
    try:
        float(n)   # Type-casting the string to `float`.
                   # If string is not a valid `float`, 
                   # it'll raise `ValueError` exception
    except ValueError:
        return False
    return True

## Product Masterfile

This one is much more straightforward to clean.

In [ ]:
product_master = pd.read_excel (r'/content/drive/MyDrive/Colab Notebooks/Data Files/SALES/ALL 2017 Product Nov rows21536 Havaianas Masterfile (Jaime No Blanks 1) Nov 2017 with notes.xlsx',)

product_master.head()

,Product Style_Colorcode_Size,PRODUCT CODE,BARCODE,SEASON,BARCODE+SEASON,\nGROUP,STYLE,COLOR CODE,COLOR,SIZE,...,EXTRA,PRINT,COLOR FAMILY,SIMILAR TO STYLE,PRINT TYPE,EMBELLISHMENT,STRAP,LAUNCH STATUS,2017 LAUNCH,PHOTO ID
0,ALLEGRA_0154_356,4135953.0,7.891225e+12,2017.0,78912245541332017,WMN,ALLEGRA,154.0,SAND GREY,356.0,...,NaN,PLAIN,BEIGE,ALLEGRA,NOT PRINTED,NONE,ALLEGRA STYLE STRAP,FOR WHOLESALES,NaN,ALLEGRA_0154
1,ALLEGRA_0154_378,4135953.0,7.891225e+12,2017.0,78912245541402017,WMN,ALLEGRA,154.0,SAND GREY,378.0,...,NaN,PLAIN,BEIGE,ALLEGRA,NOT PRINTED,NONE,ALLEGRA STYLE STRAP,FOR WHOLESALES,NaN,ALLEGRA_0154
2,ALLEGRA_0154_390,4135953.0,7.891225e+12,2017.0,78912245541572017,WMN,ALLEGRA,154.0,SAND GREY,390.0,...,NaN,PLAIN,BEIGE,ALLEGRA,NOT PRINTED,NONE,ALLEGRA STYLE STRAP,FOR WHOLESALES,NaN,ALLEGRA_0154
3,FLAT STYLE_0121_334,NaN,7.895266e+12,2016.0,78952659345652016,WMN,FLAT STYLE,121.0,BEIGE,334.0,...,NaN,PRINTED,BEIGE,FLAT,ABSTRACT,YES,FLAT STYLE STRAP,DISREGARD Column,DISREGARD Column,FLAT STYLE_0121
4,FLAT STYLE_0121_356,NaN,7.895266e+12,2016.0,78952659345722016,WMN,FLAT STYLE,121.0,BEIGE,356.0,...,NaN,PRINTED,BEIGE,FLAT,ABSTRACT,YES,FLAT STYLE STRAP,DISREGARD Column,DISREGARD Column,FLAT STYLE_0121


## Receipt Reports

This one is much more complicated. First I'll load the full table, no column names, and then separate transaction details and item details, to be cross-identified with the receipt number as key.

In [ ]:
sales_raw = pd.read_excel (r'/content/drive/MyDrive/Colab Notebooks/Data Files/SALES/DOM 2017 Sales Apr 2082rows Copy of Sales Report by Receipt with Item Details with notes.xlsx',
                              skiprows = 12, header=None)

sales_raw.head()

,0,1,2,3,4,5,6,7,8,9
0,310214.0,2017-07-04 00:00:00,ION,1.0,Regular,79.44,85.0,85.00,Female 19-26,NaN
1,NaN,23095.0,4562130119756.0,ANELLO,ANE AT-B0197B BACKPK24/35/14,SAX,FS,NaN,1.0,85.0
2,309872.0,2017-02-04 00:00:00,ION,1.0,Regular,40.09,42.9,42.90,Male 19-26,NaN
3,NaN,531.0,7891109342497.0,HAVAIANAS,BRASIL LOGO,BLACK,390.0,NaN,1.0,42.9
4,310287.0,2017-08-04 00:00:00,ION,4.0,Regular,152.94,163.64,163.64,Female 36 above,NaN


In [ ]:
sales_transactions = pd.DataFrame()
sales_items = pd.DataFrame()

In [ ]:
i = 0
receipt = ''
item_num = 0

while i < len(sales_raw):
  this_row = sales_raw.loc[i,:].copy(deep = True)
  num_items = this_row[3]

  if(is_number(num_items)):
    receipt = this_row[0]
    item_num = 0
    sales_transactions = sales_transactions.append(this_row, ignore_index = True)
  else:
    item_num = item_num + 1
    this_row[0] = receipt
    this_row[7] = item_num
    sales_items = sales_items.append(this_row, ignore_index = True)

  i = i + 1

In [ ]:
sales_transactions.columns = ['receipt','date','store','quantity','type','amount_no_gst','amount_with_gst','amount_total','gender_age','unnamed']
sales_transactions = sales_transactions.drop(['unnamed'], axis = 1)
sales_transactions.head()

,receipt,date,store,quantity,type,amount_no_gst,amount_with_gst,amount_total,gender_age
0,310214.0,2017-07-04,ION,1.0,Regular,79.44,85.00,85.00,Female 19-26
1,309872.0,2017-02-04,ION,1.0,Regular,40.09,42.90,42.90,Male 19-26
2,310287.0,2017-08-04,ION,4.0,Regular,152.94,163.64,163.64,Female 36 above
3,310202.0,2017-07-04,ION,1.0,Regular,55.98,59.90,59.90,Male 27-35
4,310029.0,2017-05-04,ION,1.0,Regular,55.98,59.90,59.90,Female 27-35


In [ ]:
sales_items.columns = ['receipt', 'item_no','barcode','brand','style','color','size','item_num','item_quantity','total']
sales_items.head()

,receipt,item_no,barcode,brand,style,color,size,item_num,item_quantity,total
0,310214.0,23095.0,4.562130e+12,ANELLO,ANE AT-B0197B BACKPK24/35/14,SAX,FS,1.0,1.0,85.00
1,309872.0,531.0,7.891109e+12,HAVAIANAS,BRASIL LOGO,BLACK,390.0,1.0,1.0,42.90
2,310287.0,4301.0,7.890732e+12,HAVAIANAS,SLIM,BLACK,378.0,1.0,2.0,31.92
3,310287.0,23514.0,7.891225e+12,HAVAIANAS,CASUAL,GREEN OLIVE,434.0,2.0,1.0,49.90
4,310287.0,23994.0,7.891225e+12,HAVAIANAS,SLIM NAUTICAL,NAVY BLUE,378.0,3.0,1.0,49.90


## POS Flags

Cleaning this takes the same process.

In [ ]:
flags_raw = pd.read_excel (r'/content/drive/MyDrive/Colab Notebooks/Data Files/SALES/DOM 2017 Sales Apr Ion  rows2081 Copy of POS Flags (Gender and Age) with Purchase Item Details with notes.xlsx',
                              skiprows = 12, header=None)

In [ ]:
flags_transactions = pd.DataFrame()
flags_items = pd.DataFrame()

i = 0
receipt = ''
item_num = 0

while i < len(flags_raw):
  this_row = flags_raw.loc[i,:].copy(deep = True)
  num_items = this_row[5]

  if(is_number(num_items)):
    receipt = this_row[1]
    item_num = 0
    flags_transactions = flags_transactions.append(this_row, ignore_index = True)
  else:
    item_num = item_num + 1
    this_row[0] = receipt
    this_row[10] = item_num
    flags_items = flags_items.append(this_row, ignore_index = True)

  i = i + 1

In [ ]:
flags_transactions.columns = ['store','receipt','date','time','total','quantity','know_how','gender_age','often_buy','unnamed']
flags_transactions = flags_transactions.drop(['unnamed'], axis = 1)
flags_transactions.head()

,store,receipt,date,time,total,quantity,know_how,gender_age,often_buy
0,ION,310214.0,2017-07-04,8:37p,85.0,1.0,Brand's popular,Female 19-26,As & when
1,ION,310478.0,2017-10-04,3:18p,39.9,1.0,Brand's popular,Female 19-26,As & when
2,ION,310191.0,2017-07-04,3:16p,39.9,1.0,Brand's popular,Female 19-26,As & when
3,ION,309691.0,2017-01-04,1:38p,13.0,1.0,Brand's popular,Female 19-26,As & when
4,ION,309886.0,2017-03-04,11:47a,90.0,1.0,Brand's popular,Female 19-26,If I need a new


In [ ]:
flags_items.columns = ['receipt','color_code','size','style','item_quantity','color','group','season','brand','srp','item_num']
flags_items.head()

,receipt,color_code,size,style,item_quantity,color,group,season,brand,srp,item_num
0,310214.0,SAX,FS,ANE AT-B0197B BACKPK24/35/14,1.0,SAX,BAGS,2016.0,ANELLO,85.0,1.0
1,310478.0,90.0,378.0,SLIM,1.0,BLACK,SLIM,2011.0,HAVAIANAS,39.9,1.0
2,310191.0,90.0,356.0,SLIM,1.0,BLACK,SLIM,2011.0,HAVAIANAS,39.9,1.0
3,309691.0,6002.0,36-40,HS MINI DIAMOND LINER SOCK,1.0,MDI06-6002,NOVELTY,2017.0,HAPPY SOCKS,13.0,1.0
4,309886.0,WI,FS,ANE AT-B0193A BACKPK 27/40/17,1.0,WI,BAGS,2016.0,ANELLO,90.0,1.0


# Combine Contents

It's much more useful to consider the transactions and items data together for both flags and sales sets.

In [ ]:
sales = sales_transactions.merge(sales_items, left_on='receipt', right_on='receipt')
flags = flags_transactions.merge(flags_items, left_on='receipt', right_on='receipt')

In [ ]:
sales.head()

,receipt,date,store,quantity,type,amount_no_gst,amount_with_gst,amount_total,gender_age,item_no,barcode,brand,style,color,size,item_num,item_quantity,total
0,310214.0,2017-07-04,ION,1.0,Regular,79.44,85.00,85.00,Female 19-26,23095.0,4.562130e+12,ANELLO,ANE AT-B0197B BACKPK24/35/14,SAX,FS,1.0,1.0,85.00
1,309872.0,2017-02-04,ION,1.0,Regular,40.09,42.90,42.90,Male 19-26,531.0,7.891109e+12,HAVAIANAS,BRASIL LOGO,BLACK,390.0,1.0,1.0,42.90
2,310287.0,2017-08-04,ION,4.0,Regular,152.94,163.64,163.64,Female 36 above,4301.0,7.890732e+12,HAVAIANAS,SLIM,BLACK,378.0,1.0,2.0,31.92
3,310287.0,2017-08-04,ION,4.0,Regular,152.94,163.64,163.64,Female 36 above,23514.0,7.891225e+12,HAVAIANAS,CASUAL,GREEN OLIVE,434.0,2.0,1.0,49.90
4,310287.0,2017-08-04,ION,4.0,Regular,152.94,163.64,163.64,Female 36 above,23994.0,7.891225e+12,HAVAIANAS,SLIM NAUTICAL,NAVY BLUE,378.0,3.0,1.0,49.90


In [ ]:
flags.head()

,store,receipt,date,time,total,quantity,know_how,gender_age,often_buy,color_code,size,style,item_quantity,color,group,season,brand,srp,item_num
0,ION,310214.0,2017-07-04,8:37p,85.0,1.0,Brand's popular,Female 19-26,As & when,SAX,FS,ANE AT-B0197B BACKPK24/35/14,1.0,SAX,BAGS,2016.0,ANELLO,85.0,1.0
1,ION,310478.0,2017-10-04,3:18p,39.9,1.0,Brand's popular,Female 19-26,As & when,90.0,378.0,SLIM,1.0,BLACK,SLIM,2011.0,HAVAIANAS,39.9,1.0
2,ION,310191.0,2017-07-04,3:16p,39.9,1.0,Brand's popular,Female 19-26,As & when,90.0,356.0,SLIM,1.0,BLACK,SLIM,2011.0,HAVAIANAS,39.9,1.0
3,ION,309691.0,2017-01-04,1:38p,13.0,1.0,Brand's popular,Female 19-26,As & when,6002.0,36-40,HS MINI DIAMOND LINER SOCK,1.0,MDI06-6002,NOVELTY,2017.0,HAPPY SOCKS,13.0,1.0
4,ION,309886.0,2017-03-04,11:47a,90.0,1.0,Brand's popular,Female 19-26,If I need a new,WI,FS,ANE AT-B0193A BACKPK 27/40/17,1.0,WI,BAGS,2016.0,ANELLO,90.0,1.0


I also want to produce a combined dataset called `sales` that combines information from both datasets.



In [ ]:
receipts = flags.loc[:,['receipt','item_num','time','know_how','often_buy','color','color_code','style','group','season','brand','srp']].merge(
    sales, left_on=['receipt','item_num','color'], right_on=['receipt','item_num','color']
)

receipts.head()

,receipt,item_num,time,know_how,often_buy,color,color_code,style_x,group,season,...,amount_with_gst,amount_total,gender_age,item_no,barcode,brand_y,style_y,size,item_quantity,total
0,310214.0,1.0,8:37p,Brand's popular,As & when,SAX,SAX,ANE AT-B0197B BACKPK24/35/14,BAGS,2016.0,...,85.0,85.0,Female 19-26,23095.0,4.562130e+12,ANELLO,ANE AT-B0197B BACKPK24/35/14,FS,1.0,85.0
1,310478.0,1.0,3:18p,Brand's popular,As & when,BLACK,90.0,SLIM,SLIM,2011.0,...,39.9,39.9,Female 19-26,4301.0,7.890732e+12,HAVAIANAS,SLIM,378.0,1.0,39.9
2,310191.0,1.0,3:16p,Brand's popular,As & when,BLACK,90.0,SLIM,SLIM,2011.0,...,39.9,39.9,Female 19-26,4300.0,7.890732e+12,HAVAIANAS,SLIM,356.0,1.0,39.9
3,309691.0,1.0,1:38p,Brand's popular,As & when,MDI06-6002,6002.0,HS MINI DIAMOND LINER SOCK,NOVELTY,2017.0,...,13.0,13.0,Female 19-26,26600.0,7.333102e+12,HAPPY SOCKS,HS MINI DIAMOND LINER SOCK,36-40,1.0,13.0
4,309886.0,1.0,11:47a,Brand's popular,If I need a new,WI,WI,ANE AT-B0193A BACKPK 27/40/17,BAGS,2016.0,...,90.0,90.0,Female 19-26,21586.0,4.582136e+12,ANELLO,ANE AT-B0193A BACKPK 27/40/17,FS,1.0,90.0


In [ ]:
receipts.to_excel(r'/content/drive/MyDrive/Colab Notebooks/Data Files/Cleaned/receipts.xls', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.
